In [2]:
import pathlib

DIR = pathlib.Path("inputs")
INPUT = DIR / "14.txt"

DATA = INPUT.read_text()

In [3]:
HEIGHT = 103
WIDTH = 101


In [4]:
import re
from dataclasses import dataclass


ROBOT_INIT_RE = re.compile(r"p=(\d+),(\d+) v=(-?\d+),(-?\d+)")

In [5]:
Point = tuple[int, int]


def sub(p1: Point, p2: Point) -> Point:
    return p1[0] - p2[0], p1[1] - p2[1]


def add(p1: Point, p2: Point) -> Point:
    return p1[0] + p2[0], p1[1] + p2[1]


@dataclass
class Robot:
    pos: Point
    vel: Point

    @staticmethod
    def wrap_pos(pos: Point) -> Point:        
        return pos[0] % WIDTH, pos[1] % HEIGHT

    def step(self):
        self.pos = self.wrap_pos(add(self.pos, self.vel))

    @classmethod
    def from_line(cls, line: str) -> "Robot":
        match = ROBOT_INIT_RE.match(line)
        assert match
        p_v_str = match.groups()
        return Robot(
            (int(p_v_str[0]), int(p_v_str[1])), (int(p_v_str[2]), int(p_v_str[3]))
        )


In [6]:
def matrix_from_robots(robots: list[Robot]) -> list[list[int]]:
    matrix = [[0] * WIDTH for _ in range(HEIGHT)]
    for robot in robots:
        matrix[robot.pos[1]][robot.pos[0]] += 1
    return matrix


def step_all(robots: list[Robot], n_times: int):
    for _ in range(n_times):
        for robot in robots:
            robot.step()


In [7]:
import numpy as np


def safety_factor(matrix: list[list[int]]) -> int:
    matrix_np = np.array(matrix)
    mid_x = WIDTH // 2
    mid_y = HEIGHT // 2
    tl = matrix_np[:mid_y, :mid_x]
    tr = matrix_np[:mid_y, mid_x + 1 :]
    bl = matrix_np[mid_y + 1 :, :mid_x]
    br = matrix_np[mid_y + 1 :, mid_x + 1 :]
    return tl.sum() * tr.sum() * bl.sum() * br.sum()


In [8]:
robots = [Robot.from_line(line) for line in DATA.split("\n")]

In [9]:
matrix = matrix_from_robots(robots)


In [10]:
step_all(robots, 100)

In [11]:
matrix_stepped = matrix_from_robots(robots)

In [12]:
safety_factor(matrix_stepped)

221655456

In [13]:
robots = [Robot.from_line(line) for line in DATA.split("\n")]

In [14]:
all_matricies = [matrix_from_robots(robots)]

for _ in range(10_000):
    step_all(robots, 1)
    all_matricies.append(matrix_from_robots(robots))

In [15]:
all_matricies_np = np.array(all_matricies)

In [16]:
def most_dense_vertical(matrix_np: np.ndarray) -> int:
    res = 0
    for y_idx in range(WIDTH):
        v_slice = matrix_np[:, y_idx]
        res = max((v_slice > 0).sum(), res)
    return res

In [18]:
counts = [most_dense_vertical(matrix) for matrix in all_matricies_np]

In [21]:
np.argmax(counts)

7858